In [58]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt

## 导入sklearn自带数据集

In [215]:
loaded_data = datasets.load_breast_cancer() # for binary classification
# loaded_data = datasets.load_iris() # for classification
# loaded_data = datasets.load_boston() # for regression

data_X = loaded_data.data
data_y = loaded_data.target

In [216]:
X_train, X_test, y_train, y_test = train_test_split(data_X, data_y, test_size=0.2, random_state=0)
print(X_train.shape,y_train.shape)

(455, 30) (455,)


## 数据和标签合并

In [217]:
train_data=np.hstack( [X_train,y_train.reshape(y_train.shape[0],1)])
test_data=np.hstack( [X_test,y_test.reshape(y_test.shape[0],1)])
print(train_data.shape)

(455, 31)


## 决策树

In [207]:
class DecisionTree(object):
    def __init__(self,data,treeType=0,min_E=.1,min_N=4):
        ''' 
        @ param: data 数据集(带标签在最后一列)
        @ param: treeType = {0,1} // CART(regression) CART(classification)
        @ param: min_E 最小下降误差
        @ param: min_N 最小划分样本数
        '''
        if int(treeType) not in [0,1]:
            raise AssertionError("treeType incorrect!")
        self.N,self.M = np.shape(data[:,:-1])
        self.treeType = treeType
        self.min_E=min_E
        self.min_N=min_N
        self.tree = self.__create_tree(data)
    
    # use for CART regression
    def __biSplitDataSet(self,dataset,idx,val):
        greater = dataset[np.nonzero(dataset[:,idx] > val)[0],:]
        lesser  = dataset[np.nonzero(dataset[:,idx] <= val)[0],:]
        return greater,lesser
    # use for CART regression
    def __err_cal(self,dataset):
        return np.var(dataset[:,-1])*np.shape(dataset)[0]
    
    # use for CART classification
    def __entro_cal(self,dataset):
        ct=Counter(dataset[:,-1])
        D=sum(ct.values())
        entro = .0
        for k,v in ct.items():
            p=v/float(D)
            entro+= -p*np.log(p) # entro type
            # entro += p*(1-p) # Gini type
        return entro
    
    # leaf of regression 均值
    def __reg_leaf(self,dataset):
        return np.mean(dataset[:,-1])

    # leaf of classification 多数表决类型
    def __clf_leaf(self,dataset):
        return int(Counter(dataset[:,-1]).most_common(1)[0][0])
    
    # main function
    def __bestSplit(self,dataset):

        min_E=self.min_E
        min_N=self.min_N
        n,m = np.shape(dataset[:,:-1])
        
        # 基于树的类型选择误差计算方式，和叶节点计算方式
        if self.treeType == 0: # regression
            err_cal = self.__err_cal
            leaf_cal= self.__reg_leaf
        else: # classification
            err_cal = self.__entro_cal__entro_cal
            leaf_cal= self.__clf_leaf
        
        # n小于最小划分样本数，直接返回叶点
        if n  < min_N:
            return None, leaf_cal(dataset)
        if len(set(dataset[:,-1])) == 1:
            return None, leaf_cal(dataset)
        
        
        baseErr = err_cal(dataset)
        minErr = baseErr+1
        bestFeatIdx = -1
        bestFeatVal = 0
        
        # 遍历特征和特征值，选择划分特征及划分点
        for idx in range(m):
            for val in set(dataset[:,idx]):
                greater,lesser = self.__biSplitDataSet(dataset,idx,val)
                if np.shape(greater)[0] < min_N or np.shape(lesser)[0] < min_N :
                    continue
                
                # regression
                if self.treeType == 0 :
                    curErr = err_cal(greater)+err_cal(lesser)
                # classification
                else:
                    curErr = (np.shape(greater)[0]/float(n))*err_cal(greater) \
                                +(np.shape(lesser)[0]/float(n))*err_cal(lesser)
                
                if curErr < minErr:
                    minErr = curErr
                    bestFeatIdx = idx
                    bestFeatVal = val
                    
        # 下降误差过小，没有划分必要
        if baseErr - minErr < min_E:
            return None, np.mean(dataset[:,-1])
        greater,lesser = self.__biSplitDataSet(dataset,bestFeatIdx,bestFeatVal)
        
        if np.shape(greater)[0] < min_N or np.shape(lesser)[0] < min_N :
            return None, leaf_cal(dataset[:,-1])
        
        return bestFeatIdx,bestFeatVal

    def __create_tree(self,dataset):
        featIdx,featVal = self.__bestSplit(dataset)
        if featIdx == None:
            return featVal
        retTree = {}
        retTree['spIdx'] = featIdx
        retTree['spVal'] = featVal
        greater,lesser = self.__biSplitDataSet(dataset,featIdx,featVal)
        retTree['left'] = self.__create_tree(lesser)
        retTree['right'] = self.__create_tree(greater)
        return retTree
    
    def __isTree(self,obj):
        return isinstance(obj,dict)
    
    def __pred_helper(self,tree,inData):
        if not self.__isTree(tree):
            return float(tree)
        if inData[tree['spIdx']]> tree['spVal']:
            return self.__pred_helper(tree['right'],inData)
        else:
            return self.__pred_helper(tree['left'],inData)
            
    def predict(self,test_data):
        yHat = np.zeros( np.shape(test_data)[0] )
        i=0
        for row in test_data:
            yHat[i]=self.__pred_helper(self.tree,row)
            i+=1
        return yHat

In [218]:
tt=DecisionTree(train_data,1,0.02,2) # 分类树 测试
yH=tt.predict(test_data)

In [222]:
err_size=len(    np.nonzero(yH!=test_data[:,-1])[0]  )
print(len(    np.nonzero(yH!=test_data[:,-1])[0]  ))
print(len(    np.nonzero(yH==test_data[:,-1])[0]  ))
print("accuracy: {:.5}".format(1- err_size/np.shape(test_data)[0]))

10
104
accuracy:0.91228
